# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Flux-Analysis-Tools" data-toc-modified-id="Flux-Analysis-Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flux Analysis Tools</a></div><div class="lev1 toc-item"><a href="#Yarrowia-SBML-model-from-Chalmers-GEM" data-toc-modified-id="Yarrowia-SBML-model-from-Chalmers-GEM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Yarrowia SBML model from <a href="https://github.com/djinnome/Yarrowia_lipolytica_W29-GEM" target="_blank">Chalmers GEM</a></a></div><div class="lev1 toc-item"><a href="#Gene-knockouts" data-toc-modified-id="Gene-knockouts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gene knockouts</a></div><div class="lev1 toc-item"><a href="#Carbon-sources" data-toc-modified-id="Carbon-sources-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Carbon sources</a></div><div class="lev2 toc-item"><a href="#Malate" data-toc-modified-id="Malate-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Malate</a></div><div class="lev2 toc-item"><a href="#Citrate" data-toc-modified-id="Citrate-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Citrate</a></div><div class="lev2 toc-item"><a href="#Glucose" data-toc-modified-id="Glucose-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Glucose</a></div><div class="lev1 toc-item"><a href="#Simulating-Carbon-sources-across-gene-knockouts" data-toc-modified-id="Simulating-Carbon-sources-across-gene-knockouts-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Simulating Carbon sources across gene knockouts</a></div><div class="lev2 toc-item"><a href="#Growth-rates" data-toc-modified-id="Growth-rates-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Growth rates</a></div><div class="lev2 toc-item"><a href="#Fluxes" data-toc-modified-id="Fluxes-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Fluxes</a></div><div class="lev1 toc-item"><a href="#Add-symbols-to-genes" data-toc-modified-id="Add-symbols-to-genes-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Add symbols to genes</a></div>

# Flux Analysis Tools

```
After some discussion with Bill and Scott, I think we’ve assembled a list of simulations to include.  Given that we need these in short order (we want the publication submitted in less than 60 days), I believe we’d be okay with an FBA model rather than including the Boltzman code. Specifically, running the Yarrowia FBA model with glucose, citrate, or malate as the carbon source in combination with wild-type and in silico deletions of idp1, zwf1, mae1. Maximum biomass production should be the objection function since we did the experiments during exponential growth phase.
 
So the 12 conditions are:
Wild-type, Glucose as sole carbon source
Idp1 deletion, Glucose as sole carbon source
Zwf1 deletion, Glucose as sole carbon source
Mae1 deletion, Glucose as sole carbon source
Wild-type, Citrate as sole carbon source
Idp1 deletion, Citrate as sole carbon source
Zwf1 deletion, Citrate as sole carbon source
Mae1 deletion, Citrate as sole carbon source
Wild-type, Malate as sole carbon source
Idp1 deletion, Malate as sole carbon source
Zwf1 deletion, Malate as sole carbon source
Mae1 deletion, Malate as sole carbon source
 
Hopefully these 12 models will yield some interesting eye candy, which can then be compared to the experimental data in our manuscript.  
 
Please let us know if you have questions.  We’d love data as soon as you have it, to synthesize with the paper.
```

In [2]:
import cobra
import cobra.io
from IPython.display import HTML, display, SVG, Markdown, Image
import os, sys
import pandas as pd
from io import StringIO
import numpy as np

#def explode(series):
#    return pd.concat([pd.DataFrame(list(v), index=np.repeat(k,len(v),axis=1)) 
#            for k,v in series.to_dict().items()
#              ])    
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

def get_producers( met, fluxes ):
    producers = []
    production_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] < 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] > 0) and (met in rxn.products)):
            producers.append( rxn )
            production_flux[rxn.id] = abs(fluxes[rxn.id])
    return producers, pd.Series(production_flux)

def get_consumers( met, fluxes ):
    consumers = []
    consumption_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] > 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] < 0) and (met in rxn.products)):
            consumers.append( rxn )
            consumption_flux[rxn.id] = abs(fluxes[rxn.id])
    return consumers, pd.Series(consumption_flux)
def summarize( met, fluxes, out=sys.stdout, use_metabolite_names=True, tol = 1e-5 ):
    ptitle = '\nProducing reactions -- {}[{}] ({})\n'.format(met.name, met.compartment, met.id)
    pcols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ptitle)
    out.write('-'*len(ptitle) + '\n')
    out.write(pcols)
    out.write('\t'.join(['-'*len(col) for col in pcols.split('\t')]) + '\n')
    producers, production_flux = get_producers( met, fluxes )
    production_pct = production_flux/production_flux.sum()*100
    production_pct
    for producer in producers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format(production_pct[producer.id], 
                                            production_flux[producer.id], 
                                            producer.id, 
                                            producer.build_reaction_string(use_metabolite_names=True),
                                            producer.build_reaction_string(use_metabolite_names=False)))
    ctitle = '\nConsuming reactions -- {}[{}] ({})\n'.format(met.name, met.compartment, met.id)
    ccols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ctitle)
    out.write('-'*len(ctitle) + '\n')
    out.write(ccols)
    out.write('\t'.join(['-'*len(col) for col in ccols.split('\t')]) + '\n')
    consumers, consumption_flux = get_consumers( met, fluxes )
    consumption_pct = consumption_flux/consumption_flux.sum()*100
    for consumer in consumers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format(consumption_pct[consumer.id], 
                                            consumption_flux[consumer.id], 
                                            consumer.id, 
                                            consumer.build_reaction_string(use_metabolite_names=True),
                                            consumer.build_reaction_string(use_metabolite_names=False)))

# Yarrowia SBML model from [Chalmers GEM](https://github.com/djinnome/Yarrowia_lipolytica_W29-GEM)

In [5]:

mdir = 'ModelFiles'
jdir = os.path.join(mdir, 'json')
xdir = os.path.join(mdir, 'xml')
os.listdir('.')
yarrowia = cobra.io.read_sbml_model(os.path.join(xdir,'iYali4.xml' ))
cobra.io.save_json_model(yarrowia, os.path.join(jdir, 'iYali4.json'))

cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xAMINOACID at 0x10ea4e3c8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xBIOMASS at 0x10eb87828>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xLIPID at 0x10eb87cf8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xMAINTENANCE at 0x10eb87908>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_EM at 0x10eb87c18>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_LP at 0x10eb87b38>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_MM at 0x10eb879b0>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_FA_EM at 0x10eb87978>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_FA_EN at 0x10eb879e8>
cobra/io/sbml3.py:

cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003509 at 0x10e71b9e8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003510 at 0x10e71b358>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003511 at 0x10e71bc18>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003512 at 0x10e71bcf8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003513 at 0x10e71bd68>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003514 at 0x10e71bdd8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003515 at 0x10e71be80>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003516 at 0x10e71beb8>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003517 at 0x10e8cb048>
cobra/io/sbml3.py:353 UserWarning: ignoring in

# Gene knockouts

In [6]:
kos = pd.DataFrame()
kos['GeneId'] =pd.Series(dict(
     MAE1 = 'YALI0E18634g',
     GDH1 = 'YALI0F17820g',
     IDP1 = 'YALI0F04095g',
     ZWF1 = 'YALI0E22649g',
     GND1 = 'YALI0B15598g',
    pos5 = 'YALI0E17963g'))
kos.index.name='GeneName'
#display(kos)

kos['GeneObj'] = kos['GeneId'].apply(yarrowia.genes.get_by_id)
kos['RxnObj'] = kos['GeneObj'].apply(lambda x: x.reactions)
kos['RxnId'] = kos['RxnObj'].apply(lambda rxns: [rxn.id for rxn in rxns][0])
kos = kos.reset_index().set_index('GeneId')
kos
#yarrowia.genes

,GeneName,GeneObj,RxnObj,RxnId
GeneId,,,,
YALI0F17820g,GDH1,YALI0F17820g,(y000471: s_0180 + s_0419 + s_0794 + s_1212 --...,y000471
YALI0B15598g,GND1,YALI0B15598g,(y000889: s_0340 + s_1207 --> s_0456 + s_0577 ...,y000889
YALI0F04095g,IDP1,YALI0F04095g,(y000661: s_0942 + s_1211 --> s_0184 + s_0462 ...,y000661
YALI0E18634g,MAE1,YALI0E18634g,(y000719: s_0068 + s_1210 --> s_0460 + s_1214 ...,y000719
YALI0E22649g,ZWF1,YALI0E22649g,(y000466: s_0568 + s_1207 --> s_0335 + s_0794 ...,y000466
YALI0E17963g,pos5,YALI0E17963g,(y000766: s_0437 + s_1200 --> s_0397 + s_0799 ...,y000766


In [6]:
for rxn in kos.loc['YALI0E17963g','RxnObj']:
    display(rxn)

Reaction identifier,y000772
Name,NADH kinase mitochondrial
Memory address,0x01126beb00
Stoichiometry,s_0437 + s_1205 --> s_0397 + s_0799 + s_1214 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y000766
Name,NAD kinase
Memory address,0x01126be710
Stoichiometry,s_0437 + s_1200 --> s_0397 + s_0799 + s_1210 ATP + NAD --> ADP + H+ + NADP(+)
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


In [7]:
yarrowia.reactions.y000770

Reaction identifier,y000770
Name,"NADH dehydrogenase, cytosolic/mitochondrial"
Memory address,0x01126bee10
Stoichiometry,s_0794 + s_1203 + s_1537 --> s_1198 + s_1535 H+ + NADH + ubiquinone-6 --> NAD + ubiquinol-6
GPR,YALI0F25135g
Lower bound,0.0
Upper bound,1000.0


In [8]:
yarrowia.reactions.y000769

Reaction identifier,y000769
Name,NAD synthase
Memory address,0x01126be3c8
Stoichiometry,s_0438 + s_0593 + s_0808 + s_1001 <=> s_0425 + s_0637 + s_0800 + s_0994 + s_1201 ATP + deamido-NAD(+) + H2O + L-glutamine <=> AMP + diphosphate + H+ + L-glutamate + NAD
GPR,YALI0A20108g
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
yarrowia.reactions.y000771

Reaction identifier,y000771
Name,NADH kinase
Memory address,0x01126bec88
Stoichiometry,s_0434 + s_1203 --> s_0394 + s_0794 + s_1212 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


In [10]:
yarrowia.reactions.y000772

Reaction identifier,y000772
Name,NADH kinase mitochondrial
Memory address,0x01126beb00
Stoichiometry,s_0437 + s_1205 --> s_0397 + s_0799 + s_1214 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


# Carbon sources


In [6]:
carbon_source_list = """
Exchange reaction	carbon source	formula	MW	carbons	mol C / L	g/L	pH
y001714	glucose	C6H12O6	180.16	6	0.6660746	20	5
y001687	citric acid	C6H8O7	192.12	6	0.6660746	21.3277087	5
y001552	malic acid	C4H6O5	134.09	4	0.6660746	22.32848579	5
"""

carbon_sources = pd.read_table(StringIO(carbon_source_list),index_col='carbon source')
carbon_sources


,Exchange reaction,formula,MW,carbons,mol C / L,g/L,pH
carbon source,,,,,,,
glucose,y001714,C6H12O6,180.16,6,0.666075,20.000000,5
citric acid,y001687,C6H8O7,192.12,6,0.666075,21.327709,5
malic acid,y001552,C4H6O5,134.09,4,0.666075,22.328486,5


## Malate

In [12]:
for rxn in yarrowia.metabolites.s_0067.reactions:
    display(rxn)

Reaction identifier,y001652
Name,alpha-ketoglutarate/malate transporter
Memory address,0x011283d6a0
Stoichiometry,s_0067 + s_0180 <=> s_0066 + s_0181 (S)-malate + 2-oxoglutarate <=> (S)-malate + 2-oxoglutarate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Reaction identifier,y001552
Name,(S)-malate exchange
Memory address,0x011281c6d8
Stoichiometry,s_0067 --> (S)-malate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001901
Name,L-malate transport
Memory address,0x0112895ef0
Stoichiometry,s_0067 <=> s_0066 (S)-malate <=> (S)-malate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Citrate

In [13]:
for rxn in yarrowia.metabolites.s_0523.reactions:
    display(rxn)

Reaction identifier,y001687
Name,citrate(3-) exchange
Memory address,0x011284f320
Stoichiometry,s_0523 --> citrate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001686
Name,citrate transport
Memory address,0x011284f2b0
Stoichiometry,s_0523 <=> s_0522 citrate <=> citrate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Glucose

In [14]:
for rxn in yarrowia.metabolites.s_0565.reactions:
    display(rxn)
 

Reaction identifier,y001714
Name,D-glucose exchange
Memory address,0x011284fcc0
Stoichiometry,s_0565 <-- D-glucose <--
GPR,
Lower bound,-1.0
Upper bound,-1.0


Reaction identifier,y000370
Name,"exo-1,3-beta-glucan glucohydrase"
Memory address,0x01125d49b0
Stoichiometry,s_0003 + s_0805 --> s_0565 (1-3)-beta-D-glucan + H2O --> D-glucose
GPR,YALI0B03564g or YALI0B17996g or YALI0F05390g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001166
Name,glucose transport
Memory address,0x01127ac048
Stoichiometry,s_0565 --> s_0563 D-glucose --> D-glucose
GPR,YALI0A14212g or YALI0B00396g or YALI0F19184g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001024
Name,sucrose hydrolyzing enxyme
Memory address,0x011273def0
Stoichiometry,s_0805 + s_1466 --> s_0554 + s_0565 H2O + sucrose --> D-fructose + D-glucose
GPR,YALI0E26719g
Lower bound,0.0
Upper bound,1000.0


# Simulating Carbon sources across gene knockouts

## Growth rates

In [7]:
glucose_exchange = 'y001714'
growth_rates = pd.DataFrame()
fluxes = pd.DataFrame()
wild_type = pd.Series()
for carbon_source in carbon_sources.index:  
    with yarrowia:
        yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = 0
        yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = -1
        wild_type[carbon_source] = yarrowia.optimize().objective_value
        growth_rates[carbon_source] = cobra.flux_analysis.single_gene_deletion(yarrowia,gene_list = kos.index)['growth']
        
growth_rates.index = [list(gene)[0] for gene in growth_rates.index]
growth_rates.index.name = 'GenotypeId'
growth_rates.columns.name = 'Carbon source'
growth_rates['GenotypeName'] = kos['GeneName']

growth_rates.loc['Wild-type'] = wild_type
growth_rates = growth_rates.reset_index().set_index(['GenotypeId','GenotypeName'])
growth_rates.to_csv('Yarrowia_KO_x_carbon_source.csv')
HTML(growth_rates.round(3).to_html())

,Carbon source,glucose,citric acid,malic acid
GenotypeId,GenotypeName,,,
YALI0E17963g,pos5,0.106,0.075,0.051
YALI0B15598g,GND1,0.105,0.075,0.050
YALI0E22649g,ZWF1,0.105,0.075,0.050
YALI0E18634g,MAE1,0.104,0.073,0.049
YALI0F04095g,IDP1,0.106,0.074,0.051
YALI0F17820g,GDH1,0.103,0.073,0.049
Wild-type,NaN,0.106,0.075,0.051


## Fluxes

In [26]:
glucose_exchange = 'y001714'
idx = pd.MultiIndex.from_product([carbon_sources.index, ['wild-type'] + list(kos['GeneName'].values)],names=['carbon source','genotype']  )
#display(idx)
fluxes = pd.DataFrame(columns=idx)
reduced_costs = pd.DataFrame(columns=idx)

for carbon_source in carbon_sources.index:  
    yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = 0
    yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = -100
    solution = yarrowia.optimize().to_frame()
    fluxes[carbon_source, 'wild-type'] = solution['fluxes']
    reduced_costs[carbon_source, 'wild-type'] = solution['reduced_costs']
    for ko in kos.index:
        with yarrowia:
            yarrowia.genes.get_by_id(ko).knock_out()
            solution = yarrowia.optimize().to_frame()
            fluxes[carbon_source, kos.loc[ko,'GeneName']] = solution['fluxes']
            reduced_costs[carbon_source, kos.loc[ko,'GeneName']] = solution['reduced_costs']
    if carbon_source != 'glucose':
        yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = -100
        yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = 0    
fluxes['RxnName'] = [yarrowia.reactions.get_by_id(rxn).name for rxn in fluxes.index]
reduced_costs['RxnName'] = [yarrowia.reactions.get_by_id(rxn).name for rxn in reduced_costs.index]
#fluxes['Subsystem'] = [yarrowia.reactions.get_by_id(rxn).notes['SUBSYSTEM'] if 'SUBSYSTEM' in yarrowia.reactions.get_by_id(rxn).notes else '' for rxn in fluxes.index ]
#reduced_costs['Subsystem'] = [yarrowia.reactions.get_by_id(rxn).notes['SUBSYSTEM'] if 'SUBSYSTEM' in yarrowia.reactions.get_by_id(rxn).notes else '' for rxn in reduced_costs.index ]

fluxes.index.name='RxnId'


reduced_costs.index.name='RxnId'
fluxes = fluxes.reset_index().set_index(['RxnId', 'RxnName'])
reduced_costs = reduced_costs.reset_index().set_index(['RxnId', 'RxnName'])
fluxes.to_csv('Yarrowia_fluxes.csv')
reduced_costs.to_csv('Yarrowia_reduced_costs.csv')
display(HTML(fluxes.round(3).to_html()))
display(reduced_costs.round(3))

carbon source                                                     glucose  \
genotype                                                        wild-type   
RxnId        RxnName                                                        
xAMINOACID   Amino acid composition                                -0.000   
xBIOMASS     Biomass production                                     0.000   
xLIPID       Lipids pool                                           -0.000   
xMAINTENANCE Non-growth associated maintenance (NGAM)              -0.014   
xPOOL_AC_EM  Acyl-CoAs pool                                         0.000   
xPOOL_AC_LP  Acyl-CoAs pool                                         0.000   
xPOOL_AC_MM  Acyl-CoAs pool                                         0.000   
xPOOL_FA_EM  Fatty acids pool                                       0.000   
xPOOL_FA_EN  Fatty acids pool                                       0.000   
xPOOL_FA_LP  Fatty acids pool                                       0.000   
xPOOL_FA_MI  Fatty acids pool                                       0.000   
xPOOL_FA_MM  Fatty acids pool                                       0.000   
y000001      (R)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000002      (R)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000004      (S)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000005      1,3-beta-glucan synthase                               0.000   
y000006      1,6-beta-glucan synthase                              -0.095   
y000007      1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)...     0.000   
y000008      1-acyl-sn-gylcerol-3-phosphate acyltransferase        -0.000   
y000009      1-acyl-sn-gylcerol-3-phosphate acyltransferase         0.000   
y000010      1-phosphatidylinositol-3-phosphate 5-kinase           -0.012   
y000012      1-pyrroline-5-carboxylate dehydrogenase                0.000   
y000013      2,3-diketo-5-methylthio-1-phosphopentane degrad...     0.000   
y000014      2,5-diamino-6-ribitylamino-4(3H)-pyrimidinone 5...     0.000   
y000015      2,5-diamino-6-ribosylamino-4(3H)-pyrimidinone 5...    -0.000   
y000016      2-aceto-2-hydroxybutanoate synthase                    0.000   
y000017      2-amino-4-hydroxy-6-hydroxymethyldihydropteridi...    -0.007   
y000018      2-aminoadipate transaminase                            0.000   
y000019      2-dehydropantoate 2-reductase                          0.000   
y000020      2-deoxy-D-arabino-heptulosonate 7-phosphate syn...    -0.000   
...                                                                   ...   
y300052      glutaryl-CoA dehydrogenase                            -0.019   
y300054      crotonyl-CoA hydratase                                 0.000   
y300055      (S)-3-Hydroxybutanoyl-CoA oxidoreductase               0.000   
y300056      cellobiose glucohydrolase                              0.000   
y300057      L-Glutamate 5-semialdehyde:NAD+ oxidoreductase        -0.000   
y300058      5-oxo-L-proline amidohydrolase (ATP-hydrolysing)       0.000   
y300059      succinyl-CoA:acetoacetate CoA-transferase             -0.000   
y300060      GTP diphosphohydrolase (diphosphate-forming);         -0.014   
y300061      GTP diphosphate-lyase (cyclizing; 3',5'-cyclic-...    -0.000   
y300062      L-aspartate 1-carboxy-lyase (beta-alanine-forming)    -0.049   
y300063      glutathione gamma-glutamylaminopeptidase              -0.000   
y300064      3'-Phospho-5'-adenylyl sulfate 3'-phosphohydrolase    -0.012   
y300065      L-Serine:pyruvate aminotransferase                     0.000   
y300066      L-Serine:glyoxylate aminotransferase                  -0.000   
y300067      serine racemase                                       -0.000   
y300068      L-Serine hydro-lyase                                  -0.051   
y300069      Uridine triphosphate pyrophosphohydrolase             -0.014   
y300070      Inosine 5'-triphosphate pyrophosphohydrolase          -0.002

In [47]:
gfluxes = pd.Series([yarrowia.reactions.get_by_id(rxnId).genes for rxnId, rxnName in fluxes.index],index=fluxes.index)# [rxnId for rxnId, rxnName in fluxes.index])

explode(gfluxes)


AxisError: axis 1 is out of bounds for array of dimension 1

In [23]:
k, v

(('y000001', '(R)-lactate:ferricytochrome-c 2-oxidoreductase'),
 frozenset({<Gene YALI0C06446g at 0x10f34cb00>,
            <Gene YALI0D09273g at 0x10f36a080>,
            <Gene YALI0E03212g at 0x10f3ef630>}))

# Add symbols to genes


In [21]:
import pandas as pd
proteins = pd.read_table('yarrowia/avg_protein_abundance.txt',index_col='gene')
proteins[proteins['pathway'].str.contains(r'nitrogen|pentose phosphate|citric acid cycle|glycolysis|PPP')]

,symbol,pathway,Cit_idp,Cit_mae,Cit_WT,Cit_zwf,Glu_idp,Glu_mae,Glu_WT,Glu_zwf,Mal_idp,Mal_mae,Mal_WT,Mal_zwf
gene,,,,,,,,,,,,,,
YALI0D09361g,ACO1,citric acid cycle,74.446667,69.831667,56.690000,50.938333,85.488333,85.150000,74.923333,39.193333,135.201667,43.688333,54.481667,61.476667
YALI0E00638g,CIT1,citric acid cycle,130.285000,128.388333,132.106667,93.611667,102.581667,140.891667,148.475000,68.400000,91.938333,8.171667,64.815000,38.155000
YALI0E02684g,CIT2,citric acid cycle,47.720000,67.676667,60.515000,34.350000,48.258333,82.786667,88.900000,33.538333,89.101667,40.323333,68.751667,51.346667
YALI0C06776g,FUM1,citric acid cycle,26.165000,36.101667,32.863333,22.515000,29.866667,31.196667,31.601667,18.200000,62.038333,27.496667,36.745000,39.255000
YALI0E05137g,IDH1,citric acid cycle,32.436667,43.623333,34.053333,30.973333,71.000000,58.091667,57.841667,30.518333,65.535000,44.126667,53.658333,34.775000
YALI0D06303g,IDH2,citric acid cycle,22.601667,35.640000,32.863333,18.871667,65.163333,58.313333,72.245000,16.088333,32.403333,7.773333,36.901667,10.388333
YALI0F04095g,IDP1,citric acid cycle,0.040000,66.350000,47.326667,24.016667,0.116667,21.573333,26.948333,27.550000,0.283333,52.923333,53.761667,62.566667
YALI0E24013g,ISC1,citric acid cycle,33.313333,70.426667,44.888333,25.588333,24.193333,18.883333,29.233333,16.126667,23.325000,8.138333,32.206667,13.618333
YALI0D04741g,ISC2,citric acid cycle,4.110000,12.551667,7.385000,3.155000,7.858333,5.195000,7.706667,4.491667,7.870000,1.566667,5.650000,3.493333


In [2]:
kos = """YALI0F04095g
YALI0E18634g
YALI0F17820g
YALI0E22649g
YALI0B15598g
YALI0E17963g""".split('\n')
